In [1]:
#Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, TimestampType


In [2]:
#Criação da sessão do spark
spark = SparkSession.builder \
    .appName("KafkaToParquet") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
    .getOrCreate()


In [3]:
#Definição schema esperado
schema = StructType() \
    .add("id", StringType()) \
    .add("user_id", StringType()) \
    .add("name", StringType()) \
    .add("description", StringType()) \
    .add("price", StringType()) \
    .add("department", StringType()) \
    .add("image", StringType()) \
    .add("createdAt", TimestampType())


In [9]:
"""
Leitura continua do kafka

-conecta com broker
-escuta o topico "products"
-caputra apenas das mensagens mais recentes
"""
raw_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "products") \
    .option("startingOffsets", "latest") \
    .load()
    
"""
-Tranformamos em texto os arquivo binarios que vem do kafka
-interpretamos como json e aplica o schema
-extrai cada campo do json como a coluna do dataframe definido
"""
json_df = raw_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), schema).alias("data")) \
    .select("data.*")


"""
# Bonus 1 : validacao
"""
filter_df = json_df\
.filter(col("price").cast("double") > 1.0)


"""
Escrita continua
-define a saida sera continua em streaming
-grava em formato parquet
-usamos o checkpointLocation para controlar o processo e evitar duplicações
"""
query = filter_df.writeStream \
    .format("parquet") \
    .outputMode("append") \
    .option("path", "/home/jovyan/work/parquet_output") \
    .option("checkpointLocation", "/home/jovyan/work/checkpoint") \
    .start()


"""
Espera do processo terminar
- mantem o processo rodando infinitamente até cancelarmos manualmente
"""
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 